# COVID-19_simulator
2020-03-21 yasubei (@yasubeitwi)

## 内容
- コロナウイルス（COVID-19）のシミュレータです。
- 以下の記事に触発されて作りました。<br>
　コロナウイルスなどのアウトブレイクは、なぜ急速に拡大し、どのように「曲線を平にする」ことができるのか<br>
　https://www.washingtonpost.com/graphics/2020/health/corona-simulation-japanese/ <br>

- 別にUnityでも良いんですけど「可視化を学んでいる学生さんにも使って欲しいな」という思いでJupyterLabで作りました。

## インストール
- ipycanvasというパッケージを使っています。
- ipycanvasですが、Docker(kaggle-image)上ではうまくインストールすることが出来なかったので、Macで動作確認しています。
- インストール方法について詳しくは01_ipycanvas_basic_usageを参照ください。

## 対象ファイル
- 特にありません。

## 出力先
- ファイルは特に出力しません。可視化を行うのみです。

## トラブルシューティング
- zmq.error.ZMQError: Too many open files のエラーが出た時は以下の手順で対処してみてください <br>
  - 以下のコマンドでfdの最大数を確認する（私は256でした） <br>
    ulimit -Sn <br>
  - 以下のコマンドでfdの最大数を変更する（私は1024にするとhuman_count=150個以上で動作可能になりました） <br>
    ulimit -Sn 1024 <br>

## 準備

In [1]:
# 必要なライブラリ読み込み
import time
import numpy as np
from threading import Timer
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display as IPython_display
from ipycanvas import Canvas, hold_canvas
import math
from math import pi

In [2]:
# タイマークラス
class RepeatedTimer(Timer):
    def __init__(self, interval, function, args=[], kwargs={}):
        Timer.__init__(self, interval, self.run, args, kwargs)
        self.thread = None
        self.function = function

    def run(self):
        self.thread = Timer(self.interval, self.run)
        self.thread.start()
        self.function(*self.args, **self.kwargs)

    def cancel(self):
        if self.thread is not None:
            self.thread.cancel()
            self.thread.join()
            del self.thread

In [3]:
# ゲームオブジェクトクラス
class GameObject2D:
    def __init__(self, init_x=0, init_y=0, width=0, height=0, color='black', vec_x=0, vec_y=0):
        self.init_x = init_x
        self.init_y = init_y
        self.pos_x = init_x
        self.pos_y = init_y
        self.width = width
        self.height = height
        self.color = color
        self.vec_x = vec_x
        self.vec_y = vec_y

    def update(self):
        pass

In [4]:
# 健康状態
from enum import Enum

class Health(Enum):
    FINE = 0        # 健康
    ACTIVE = 1      # 感染
    RECOVERD = 2    # 回復
    DEATH = 3       # 死亡

In [9]:
# 人間クラス
class Human(GameObject2D):
    def __init__(self, simulator, init_x=0, init_y=0, width=0, vec_x=0, vec_y=0, health=Health.FINE, recover_time=10.0):
        GameObject2D.__init__(self, init_x, init_y, width, width, self._color(health), vec_x, vec_y) # ボールはwidthだけ使用するのでheightにもwidthを入れる
        self.half_w = self.width
        self.health = health
        self.simulator = simulator
        self.recover_time = recover_time
        self.active_duration = 0

    ## 色を返す
    def _color(self, health):
        if health == Health.FINE:
            return '#a0bfc3'
        elif health == Health.ACTIVE:
            return '#b2591c'
        elif health == Health.RECOVERD:
            return '#c47fb8'
        elif health == Health.DEATH:
            return 'black'
        
    ## 人をcanvas内に収める処理
    def _crop_canvas(self):
        canvas = self.simulator.canvas
        if self.pos_x + self.half_w > canvas.width:
            self.pos_x = canvas.width - self.half_w  # 壁にくっつける
            self.vec_x = -self.vec_x
        elif self.pos_x - self.half_w < 0:
            self.pos_x = self.half_w  # 壁にくっつける
            self.vec_x = -self.vec_x

        if self.pos_y + self.half_w > canvas.height:
            self.pos_y = canvas.height - self.half_w  # 壁にくっつける
            self.vec_y = -self.vec_y
        elif self.pos_y - self.half_w < 0:
            self.pos_y = self.half_w  # 壁にくっつける
            self.vec_y = -self.vec_y

    ## 感染確認
    def _activate_check(self, human):
        if self.health == Health.ACTIVE and human.health == Health.FINE:
            human.activate()
        elif human.health == Health.ACTIVE and self.health == Health.FINE:
            self.activate()
        
    ## 人同士の当たり判定
    def _hit_a_human(self, human):
        sub_x = human.pos_x - self.pos_x 
        sub_y = human.pos_y - self.pos_y
        sub_r = math.hypot(sub_x, sub_y)
        concat_w = self.half_w + human.half_w
        
        if sub_r < concat_w:
            vec_r = math.hypot(self.vec_x, self.vec_y)
            self._activate_check(human)
            if vec_r > 0 and sub_r > 0:
                self.pos_x = human.pos_x - (sub_x / sub_r) * concat_w  # ボールにくっつける
                self.pos_y = human.pos_y - (sub_y / sub_r) * concat_w  # ボールにくっつける
                self.vec_x = -(sub_x / sub_r) * vec_r
                self.vec_y = -(sub_y / sub_r) * vec_r
            
    ## 人同士の当たり判定
    def _hit_humans(self):
        for a_human in self.simulator.humans:
            if a_human != self:
                self._hit_a_human(a_human)

    ## 回復チェック
    def _recover_check(self, update_time):
#         print(f'update_time={update_time}, active_duration={self.active_duration}, recover_time={self.recover_time}')
        if self.health == Health.ACTIVE:
            self.active_duration += update_time
            if self.active_duration > self.recover_time:
                self.recovered()
                
    ## フレームごとの更新処理
    def update(self, update_time):
        ### 回復チェック
        self._recover_check(update_time)
            
        ### 速度を加算して位置を移動
        self.pos_x += self.vec_x
        self.pos_y += self.vec_y

        self._crop_canvas() ## 人をcanvas内に収める
        self._hit_humans() ## 人同士の当たり判定

        self.simulator.canvas.fill_style = self.color
        self.simulator.canvas.fill_arc(self.pos_x, self.pos_y, self.width, 0, 2 * pi)
    
    ## アクティブ化する
    def activate(self):
        if self.health == Health.FINE:
            self.health = Health.ACTIVE
            self.color = self._color(self.health)
    
    # 回復にする
    def recovered(self):
        if self.health == Health.ACTIVE:
            self.health = Health.RECOVERD
            self.color = self._color(self.health)

        

In [27]:
class COVID19_simulator:
    def __init__(self, canvas_w, canvas_h, ):
        # Canvas作成
        self.canvas = Canvas(width=canvas_w, height=canvas_h)
        self.canvas.stroke_rect(0, 0, self.canvas.width, self.canvas.height)
        self.humans = []
        self.last_perf_count = time.perf_counter()

    ## 毎フレームごとの更新処理
    def on_update(self, hoge):
        update_time = time.perf_counter() - self.last_perf_count
        self.last_perf_count += update_time
        with hold_canvas(self.canvas):
            self.canvas.clear_rect(0, 0, self.canvas.width, self.canvas.height)
            self.canvas.stroke_rect(0, 0, self.canvas.width, self.canvas.height)
            for a_human in self.humans:
                a_human.update(update_time)
    
    ## human_countでグリッドを作る
    def _make_humans(self, human_count, move_speed, recover_time):
        min_side = min(self.canvas.width, self.canvas.height)
        max_side = max(self.canvas.width, self.canvas.height)
        aspect = max_side / min_side
        grid_size = min_side/math.sqrt(human_count/aspect)
#         print(f'aspect={aspect}, sqrt={sqrt_human_count}')
        temp_humans = []
        for grid_x in np.arange(0,self.canvas.width, grid_size):
            for grid_y in np.arange(0,self.canvas.height, grid_size):
#                 vec_rad = 2*pi*np.random.randn() # 正規分布
                vec_rad = 2*pi*np.random.rand() # ホワイトノイズ
                temp_humans.append(Human(self,
                                         grid_x+grid_size*0.5+grid_size*np.random.randn(), # 位置は少し乱数を混ぜる
                                         grid_y+grid_size*0.5+grid_size*np.random.randn(), # 位置は少し乱数を混ぜる
                                         6,
                                         move_speed*math.cos(vec_rad),
                                         move_speed*math.sin(vec_rad),
                                        recover_time=recover_time))
        self.humans = temp_humans

    ## 最初の感染者を作る
    def _init_active(self, active_cx, active_cy):
        active_cx = self.canvas.width * 0.5
        active_cy = self.canvas.height * 0.5
        min_distance = self.canvas.width * self.canvas.height # ありえない距離で初期化
        nearest = self.humans[0]
        
        for a_human in self.humans:
            distance = math.hypot(a_human.pos_x-active_cx, a_human.pos_y-active_cy)
            if min_distance > distance:
                nearest = a_human
                min_distance = distance
        nearest.activate()

    ## 最初の感染者を作る
    def _init_no_move(self, stop_rate):
        for a_human in self.humans:
            if a_human.health == Health.FINE:    # 感染者を止めてしまうと実験にならないのでチェックする
                if np.random.rand() < stop_rate:
                    a_human.vec_x = 0
                    a_human.vec_y = 0
        
        
    ## シミュレーション開始
    def start(self, human_count, move_speed=3.0, refresh_interval=0.05, duration=15.0, active=(0.5,0.5), recover_time=10.0, stop_rate=0.5):
        self._make_humans(human_count, move_speed, recover_time) # human_countでグリッドを作る
        self._init_active(active[0],active[1])                   # 感染者初期化
        self._init_no_move(stop_rate)                            # 社会的距離戦略により動かない人間を決める
        
        self.last_perf_count = time.perf_counter()
        t = RepeatedTimer(refresh_interval, self.on_update, ["hoge"])
        t.start()
        time.sleep(duration)
        t.cancel()

In [32]:
sim = COVID19_simulator(canvas_w=800, canvas_h=400)
sim.canvas

Canvas(height=400, width=800)

In [33]:
# ２人に一人、社会的距離戦略により動かなかった場合
sim.start(human_count=200, move_speed=1.5, duration=30.0, stop_rate=0.500)

In [34]:
# ４人に一人、社会的距離戦略により動かなかった場合
sim.start(human_count=200, move_speed=1.5, duration=30.0, stop_rate=0.750)

In [35]:
# 8人に一人、社会的距離戦略により動かなかった場合
sim.start(human_count=200, move_speed=1.5, duration=30.0, stop_rate=0.875)